# Import Statements

In [63]:
import sys
import os 
import glob
import random 
import shutil
import time

if './vision_transformer' not in sys.path:
    sys.path.append('./vision_transformer')

dataset_path = './nabirds-data/nabirds'
image_path = os.path.join(dataset_path + '/images/')

import flax
import jax
import numpy as np
import optax
import tqdm

from vit_jax import checkpoint
from vit_jax import input_pipeline
from vit_jax import utils
from vit_jax import models
from vit_jax import train
from vit_jax.configs import augreg as augreg_config
from vit_jax.configs import models as models_config
from vit_jax.configs import common as common_config
import ml_collections

import self_utils

from absl import logging
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

### Set logging functions

In [64]:
pd.options.display.max_colwidth = None
logging.set_verbosity(logging.INFO)

### Set model configuration settings

In [65]:
config = common_config
config.batch = 32

### Import datasets

In [67]:
ds_train, ds_test = self_utils.tf_load_images(image_path, batch_size = config.batch, img_size=(224,224), split=0.2)

Found 48562 files belonging to 555 classes.
Using 38850 files for training.
Using 9712 files for validation.


### Download pre-trained model

In [70]:
# Download a pre-trained model.

# Note: you can really choose any of the above, but this Colab has been tested
# with the models of below selection...
model_name = 'ViT-B_32'  #@param ["ViT-B_32", "Mixer-B_16"]

if model_name.startswith('ViT'):
  ![ -e "$model_name".npz ] || gsutil cp gs://vit_models/imagenet21k/"$model_name".npz .
if model_name.startswith('Mixer'):
  ![ -e "$model_name".npz ] || gsutil cp gs://mixer_models/imagenet21k/"$model_name".npz .

assert os.path.exists(f'{model_name}.npz')

/usr/bin/bash: line 1: gsutil: command not found


AssertionError: 